In [6]:
import sys 
sys.path.append('../')

import os

PATH_METRICS_BY_USER = os.environ['METRICS_BY_USER_DATA_PATH']

PATH_TO_TEST_DATA = os.environ['SPLIT_DATA_PATH']

In [7]:
import pandas as pd
import numpy as np
from replay.metrics import NDCG, HitRate
from replay.metrics import PerUser
from clearml import Task
from preprocessing.preparation import get_last_item

In [8]:
def get_user_metrics(task_id):
    for task in task_id:
        data_name = Task.get_task(task_id=task).get_parameters_as_dict()['General']['datasets_info']['name']
        model = Task.get_task(task_id=task).get_parameters_as_dict()['General']['model']['model']
        True_data = pd.read_csv(PATH_TO_TEST_DATA + f'test_{data_name}.csv')
        Shuffle_data = pd.read_csv(Task.get_task(task_id=task).artifacts['shuffle_pred.csv'].get_local_copy())
        Origin_data = pd.read_csv(Task.get_task(task_id=task).artifacts['test_pred.csv'].get_local_copy())
        
        metrics = pd.DataFrame()
        metrics['ndcg'] = NDCG(topk=10, query_column='user_id', mode=PerUser, rating_column='prediction')(Shuffle_data, get_last_item(True_data))['NDCG-PerUser@10'].values()
        metrics['hitrate'] = HitRate(topk=10, query_column='user_id', mode=PerUser, rating_column='prediction')(Shuffle_data, get_last_item(True_data))['HitRate-PerUser@10'].values()
        metrics.to_csv(PATH_METRICS_BY_USER + f'{model}_{data_name}_shuffle.csv', index=False)
        


        metrics = pd.DataFrame()
        metrics['ndcg'] = NDCG(topk=10, query_column='user_id', mode=PerUser, rating_column='prediction')(Origin_data, get_last_item(True_data))['NDCG-PerUser@10'].values()
        metrics['hitrate'] = HitRate(topk=10, query_column='user_id', mode=PerUser, rating_column='prediction')(Origin_data, get_last_item(True_data))['HitRate-PerUser@10'].values()
        metrics.to_csv(PATH_METRICS_BY_USER + f'{model}_{data_name}_origin.csv', index=False)



In [9]:
TASK_ID = ['TASK_ID',
           'TASK_ID',
           'TASK_ID',
           'TASK_ID',
          ]

In [10]:
get_user_metrics(TASK_ID)